## Visualizing optical flow descriptors

First, we need to make sure we can import files from `python` directory in git, so we specify the path to where git repo is cloned

In [2]:
import sys
sys.path.append('./src')
from config import *

In [3]:
import mPyPl as mp
import mPyPl as mp
import mPyPl.utils.image as mpui
from mpyplx import *
from pipe import Pipe
from moviepy.editor import *
import numpy as np
import itertools
import cv2
import math
import matplotlib.pyplot as plt
import keras

In [4]:
def close_clip(video):
    video.reader.close()
    video.audio.reader.close_proc()

Those are the classes and their id's, so in the code below, index 0 refers to *no-shot*, and 1 to *shot*

In [5]:
classes = mp.get_classes(data_dir)
print(classes)

In [22]:
def flow_to_polar(flow):
    return cv2.cartToPolar(flow[..., 0], flow[..., 1])

def visualize_flow_hsv(flow):
    hsvImg = np.zeros(flow.shape[:-1]+(3,),dtype=np.uint8)
    mag, ang = flow_to_polar(flow)
    hsvImg[..., 0] = 0.5 * ang * 180 / np.pi
    hsvImg[..., 1] = 255
    hsvImg[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    return cv2.cvtColor(hsvImg, cv2.COLOR_HSV2BGR)

# Plot the flow of first 5 frames
#flow = first_frame['dflow']
#mpui.show_images([visualize_flow_hsv(x) for x in flow[0:5]])

def get_flow_descriptor(flow,nbins=100,mag_min=0,mag_max=7):
    def descr(f,abins,mbins):
        mag, ang = flow_to_polar(f)
        h1,bins1 = np.histogram(ang.ravel(),bins=abins)
        # print(mag.shape,mbins.shape)
        h2,bins2 = np.histogram(mag.ravel(),bins=mbins)
        return [[x,y] for x,y in zip(h1,np.log(1+h2))] # we take log of mag histogram to make range smaller
    abins = [i*2*math.pi/nbins for i in range(0,nbins+1)]
    mbins = np.arange(nbins+1)/nbins*(mag_max-mag_min)+mag_min
    return np.array([descr(f,abins=abins,mbins=mbins) for f in flow],dtype=np.float32)

def plot_flow_descriptor(video,fd,steps=13,skip=7):
    fig,ax = plt.subplots(steps,3)
    for i in range(steps):
        ax[i,0].imshow(video[i*skip])
        ax[i,1].plot(fd[i*skip,:,0])
        ax[i,2].plot(fd[i*skip,:,1])
    plt.show()


We load data into dictionary, with index 0 corresponding to no-shot, and 1 - to shot. We load 5 clips for each category, together with their optical flows. And we compute descriptors here.

In [12]:
data = {}
for cls in [0,1]:
    data[cls] = (
       mp.get_datastream(data_dir,ext=".resized.mp4")
     | mp.filter('class_id',lambda x: x==cls)
     | mp.take(5)
     | mp.apply('filename','clip',lambda fn: VideoFileClip(fn).fx(vfx.resize,width=64))
     | mp.apply('clip','video',lambda c: np.array(list(c.iter_frames())))
     | mp.iter('clip',close_clip)
     | mp.apply('filename','dflow',lambda x: np.load(x.replace('.resized.mp4','.optflow.npy')))
     | mp.apply_npy('dflow','flowdescr', get_flow_descriptor)
     | mp.as_list
    )

Here are first frames from those 5 videos, and corresponding optical flow. First 2 rows are for no-shot, second two - for shot

In [14]:
mpui.show_images([x['video'][0] for x in data[0]])
mpui.show_images([visualize_flow_hsv(x['dflow'][0]) for x in data[0]])
mpui.show_images([x['video'][0] for x in data[1]])
mpui.show_images([visualize_flow_hsv(x['dflow'][0]) for x in data[1]])

We plot optical flow descriptors for several frames in the video. Indices are as follows: `data[<class id>][<frame no>]`.

In [24]:
plot_flow_descriptor(data[0][2]['video'],data[0][2]['flowdescr'],steps=7,skip=10)

In [26]:
plot_flow_descriptor(data[1][2]['video'],data[1][2]['flowdescr'],steps=7,skip=10)